Import required dependencies

In [ ]:
import os
import json
import requests
import pandas as pd
import pickle           # Save and load data
import time


# Setting Pandas options.
pd.set_option("display.max_rows", 50) # How to display all rows from data frame using pandas. Setting value to None to show all rows.
pd.set_option("display.max_columns", None)
pd.set_option("display.max_info_columns", 100)
pd.set_option("display.max_info_rows", 1000000)
pd.set_option("display.precision", 2)
#pd.set_option("styler.format.precision", 2)

load config files

In [ ]:
# Load API Key and download directory from config file
CONFIG_FILE = "../config/api-call.json"

def load_config(config_path):
    """Load configuration from a JSON file."""
    with open(config_path, "r") as file:
        return json.load(file)

Config parameters for API calls

In [ ]:
# Load config values
config = load_config(CONFIG_FILE)
# print(config)
API_ENDPOINT= config["ned"]["ned_api_endpoint"]
API_KEY = config["ned"]["demo-ned-api-key"]
DOWNLOAD_DIR = config["ned"]["ned_download_dir"]

# https://api.ned.nl/v1/utilizations?point=0&type=2&granularity=3&granularitytimezone=1&classification=2&activity=1&validfrom[strictly_before]=2020-11-17&validfrom[after]=2020-11-16


# Ensure the download directory exists
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# API details (Replace with actual API endpoint)
API_URL = API_ENDPOINT
#HEADERS = {"Authorization": f"X-AUTH-TOKEN {API_KEY}"}

# Headers with X-AUTH-TOKEN
api_headers = {
    "X-AUTH-TOKEN": API_KEY,  # Replace with your actual token
    "Content-Type": "application/json"  # Optional, if sending JSON
}

Make API calls in loop

In [39]:

# Type	    What is the type of energy carrier?	0 All, 1 Wind, 2 Solar, 3 Biogas, 4 HeatPump, 8 Cofiring, 9 Geothermal, 10 Other, 11 Waste, 12 BioOil, 13 Biomass
# 14 Wood, 17 WindOffshore, 18 FossilGasPower, 19 FossilHardCoal, 20 Nuclear, 21 WastePower, 22 WindOffshoreB, 23 NaturalGas, 24 Biomethane, 25 BiomassPower
# 26 OtherPower, 27 ElectricityMix, 28 GasMix, 31 GasDistribution, 35 CHP Total, 50 SolarThermal, 51 WindOffshoreC, 53 IndustrialConsumersGasCombination
# 54 IndustrialConsumersPowerGasCombination, 55 LocalDistributionCompaniesCombination, 56 AllConsumingGas


# Define an array of n values (Custom values instead of a fixed range)
# n_values = [1, 2, 3, 4, 8, 9, 10, 11, 12, 13, 14, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 
# 31, 35, 50, 51, 53, 54, 55, 56]  # Type values

# n_values = [1, 2]  # Type values

n_values = [56]  # Type values


# Loop through the n values
for gen_type in n_values:
    #params = {"n": n}  # Pass n as a parameter
    api_params={
        "point": "0",
        "type": gen_type,
        "granularity": "4",
        "granularitytimezone": "1",
        "classification": "2",
        "activity": "1",
        "validfrom[strictly_before]": "2024-12-31",
        "validfrom[after]": "2022-01-01"
        }

    """Download a data file from an API and save it in the specified directory."""
    response = requests.get(API_URL, headers=api_headers, params=api_params)

    print("response", response.status_code)

    # Convert JSON to Pandas DataFrame
        
    if response.status_code == 200 and response.text.strip():
        # JSON in bytes format
        json_bytes = response.content

        # Convert bytes → string → dictionary
        json_dict = json.loads(json_bytes.decode("utf-8"))

        print(json_dict)

        # Convert to DataFrame
        df = pd.DataFrame.from_dict(json_dict, orient="index")

        # Convert the list of utilizations into a DataFrame
        json_data = pd.DataFrame(json_dict["hydra:member"])

        # Display the DataFrame as a table
        print("\nConverted JSON Payload to Table Format:\n")
        # print(json_data.to_string(index=False))

            
        file_path = os.path.join(DOWNLOAD_DIR, f"power-gen-type-2022-2024-{gen_type}.json")  # Change extension as needed
        with open(file_path, "wb") as file:
            file.write(response.content)
        print(f"File downloaded successfully: {file_path}")        
    else:
        print(f"Error for {gen_type} : {response.status_code} - {response.text}")
    #filtered_df = filter_response_data(response_data)
    time.sleep(1)  # Optional delay to avoid rate limits

response 200
{'@context': '/v1/contexts/Utilization', '@id': '/v1/utilizations', '@type': 'hydra:Collection', 'hydra:totalItems': 0, 'hydra:member': [], 'hydra:view': {'@id': '/v1/utilizations?activity=1&classification=2&granularity=4&granularitytimezone=1&point=0&type=56&validfrom%5Bstrictly_before%5D=2024-12-31&validfrom%5Bafter%5D=2022-01-01', '@type': 'hydra:PartialCollectionView'}, 'hydra:search': {'@type': 'hydra:IriTemplate', 'hydra:template': '/v1/utilizations{?id,id[],point,point[],type,type[],granularity,granularity[],granularitytimezone,granularitytimezone[],classification,classification[],activity,activity[],validfrom[before],validfrom[strictly_before],validfrom[after],validfrom[strictly_after],order[validfrom]}', 'hydra:variableRepresentation': 'BasicRepresentation', 'hydra:mapping': [{'@type': 'IriTemplateMapping', 'variable': 'id', 'property': 'id', 'required': False}, {'@type': 'IriTemplateMapping', 'variable': 'id[]', 'property': 'id', 'required': False}, {'@type': 'Ir

Functionl to make API calls